In [1]:
import jsonlines
import pandas as pd

In [2]:
def get_concat_supports(supports):
    concat_supports = ""
    total_supports = len(supports)
    # print(total_supports)
    for i in range(total_supports):
        supp_sent = supports[i]['text']
        
        supp_sent = supp_sent.strip()
        if supp_sent[-1] == '.':
            concat_supports = concat_supports + supp_sent + " "
        else :
            concat_supports = concat_supports + supp_sent + ". "
            
    return concat_supports.strip()

In [3]:
def change_answer(obj):
        if obj["answerKey"] == '1' or obj["answerKey"] == 1:
            obj["answerKey"] = 'A'
        elif obj["answerKey"] == '2' or obj["answerKey"] == 2:
            obj["answerKey"] = 'B'
        elif obj["answerKey"] == '3' or obj["answerKey"] == 3:
            obj["answerKey"] = 'C'
        elif obj["answerKey"] == '4' or obj["answerKey"] == 4:
            obj["answerKey"] = 'D'

In [4]:
def change_choice_labels(obj):
        if obj["question"]["choice"]["label"] == '1' or obj["question"]["choice"]["label"] == 1:
            obj["question"]["choice"]["label"] = 'A'
        elif obj["question"]["choice"]["label"] == '2' or obj["question"]["choice"]["label"] == 2:
            obj["question"]["choice"]["label"] = 'B'
        elif obj["question"]["choice"]["label"] == '3' or obj["question"]["choice"]["label"] == 3:
            obj["question"]["choice"]["label"] = 'C'
        elif obj["question"]["choice"]["label"] == '4' or obj["question"]["choice"]["label"] == 4:
            obj["question"]["choice"]["label"] = 'D'

In [5]:
def create_csv(in_file_name, out_file_name):
    # df = pd.DataFrame(columns=COLUMN_NAMES)
    # counter = 0
    dict_list = []
    with jsonlines.open(in_file_name) as reader:
        for obj in reader:
            temp_dict = {}
            temp_dict["id"] = obj["id"]
            temp_dict["question_stem"] = obj["question"]["stem"]
            temp_dict["choice_text"] = obj["question"]["choice"]["text"]
            temp_dict["hypothesis"] = obj["hypothesis"]
            # print(obj['supports'])
            temp_dict["concat_supports"] = get_concat_supports(obj['supports'])
            change_answer(obj)
            change_choice_labels(obj)
            temp_dict["choice_label"] = obj["question"]["choice"]["label"]
            temp_dict["answerKey"] = obj["answerKey"]
            dict_list.append(temp_dict)
            
     
    df = pd.DataFrame(dict_list) 
    df.to_csv(out_file_name, encoding='utf-8', index=False)

In [6]:
def get_id_list(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        id_4_list = []
        count = 0
        id_q = ''
        for obj in reader:
            if id_q != obj['id']:
                if count == 4:
                    id_4_list.append(id_q)
                    count = 1
                    id_q = obj['id']
                else:
                    count = 1
                    id_q = obj['id']
            else :
                count = count + 1
        return id_4_list

In [7]:
print(len(get_id_list("3_ARC-Challenge_Test_openie.jsonl")))
print(get_id_list("3_ARC-Challenge_Test_openie.jsonl")[:7])

1164
['Mercury_7175875', 'Mercury_SC_409171', 'Mercury_SC_408547', 'Mercury_407327', 'MCAS_2006_9_44', 'Mercury_7270393', 'MCAS_2014_5_7']


In [8]:
print(len(get_id_list("3_ARC-Challenge_Train_openie.jsonl")))
print(get_id_list("3_ARC-Challenge_Train_openie.jsonl")[:7])

1116
['Mercury_SC_415702', 'MCAS_2009_5_6516', 'Mercury_7233695', 'Mercury_7041615', 'Mercury_7041860', 'Mercury_SC_401653', 'MEA_2016_8_14']


In [9]:
print(len(get_id_list("3_ARC-Challenge_Dev_openie.jsonl")))
print(get_id_list("3_ARC-Challenge_Dev_openie.jsonl")[:7])

294
['Mercury_SC_407695', 'Mercury_7103565', 'MDSA_2009_5_16', 'Mercury_7027230', 'Mercury_SC_405487', 'Mercury_7032743', 'Mercury_7205135']


In [10]:
path_Train = "processed/Train.csv"
path_Test = "processed/Test.csv"
path_Dev = "processed/Dev.csv"

In [ ]:
with jsonlines.open("3_ARC-Challenge_Test_processed_openie.jsonl", 'w') as writer:
    with jsonlines.open("3_ARC-Challenge_Test_openie.jsonl", 'r') as reader:
        id_set = set(get_id_list())
        id_set.remove('Mercury_412681')
        id_set.remove('Mercury_402501')
        id_set.remove('Mercury_401433')
        for obj in reader:
            if obj['id'] in id_set:
                change_answer(obj)
                change_choice_labels(obj)
                writer.write(obj)

In [11]:
def get_id_with_less_supports(path):
    A = set(get_id_list(path))
    B = set()
    with jsonlines.open(path, 'r') as reader:
        
        c = 0
        for obj in reader:
            if len(obj['supports']) < 2:
                c = c + 1
                print(obj['id'])
                B.add(obj['id'])
                
    return list(A.difference(B))

In [12]:
print(get_id_with_less_supports("3_ARC-Challenge_Train_openie.jsonl")[:7])
print(len(get_id_with_less_supports("3_ARC-Challenge_Train_openie.jsonl")))

Mercury_404786
['Mercury_7233695', 'Mercury_SC_LBS10918', 'Mercury_7222355', 'Mercury_400749', 'Mercury_7228375', 'MCAS_2006_9_27', 'Mercury_SC_406541']
Mercury_404786
1115


In [13]:
print(get_id_with_less_supports("3_ARC-Challenge_Test_openie.jsonl")[:7])
print(len(get_id_with_less_supports("3_ARC-Challenge_Test_openie.jsonl")))

Mercury_412681
Mercury_412681
Mercury_412681
Mercury_412681
Mercury_402501
Mercury_401433
['Mercury_SC_400062', 'MDSA_2013_8_23', 'Mercury_7032515', 'Mercury_7216965', 'Mercury_SC_406794', 'Mercury_416633', 'Mercury_SC_401818']
Mercury_412681
Mercury_412681
Mercury_412681
Mercury_412681
Mercury_402501
Mercury_401433
1161


In [14]:
print(get_id_with_less_supports("3_ARC-Challenge_Dev_openie.jsonl")[:7])
print(len(get_id_with_less_supports("3_ARC-Challenge_Dev_openie.jsonl")))

['Mercury_7284043', 'Mercury_7092295', 'NCEOGA_2013_5_43', 'Mercury_7246925', 'MEAP_2005_5_1', 'MCAS_2014_8_17', 'Mercury_7090598']
294


In [15]:
path_Train = "processed/Train.jsonl"
path_Test = "processed/Test.jsonl"
path_Dev = "processed/Dev.jsonl"

In [ ]:
def processed_files(path_in, path_out):
    with jsonlines.open(path_out, 'w') as writer:
        with jsonlines.open(path_in, 'r') as reader:
            id_set = set(get_id_with_less_supports(path_in))
            for obj in reader:
                if obj['id'] in id_set:
                    change_answer(obj)
                    change_choice_labels(obj)
                    writer.write(obj)

In [ ]:
processed_files("3_ARC-Challenge_Train_openie.jsonl", path_Train)

In [ ]:
processed_files("3_ARC-Challenge_Test_openie.jsonl", path_Test)

In [ ]:
processed_files("3_ARC-Challenge_Dev_openie.jsonl", path_Dev)

In [ ]:
create_csv(path_Train, "processed/Train.csv")
create_csv(path_Test, "processed/Test.csv")
create_csv(path_Dev, "processed/Dev.csv")

In [33]:
df = pd.read_csv("processed/Test.csv")

In [34]:
df.head()

,answerKey,choice_label,choice_text,concat_supports,hypothesis,id,question_stem
0,A,A,dry palms,Motion produces heat when we rub the palms of ...,George wants to warm his hands quickly by rubb...,Mercury_SC_415702,George wants to warm his hands quickly by rubb...
1,A,B,wet palms,Motion produces heat when we rub the palms of ...,George wants to warm his hands quickly by rubb...,Mercury_SC_415702,George wants to warm his hands quickly by rubb...
2,A,C,palms covered with oil,Heat up the oil a bit in the microwave and rub...,George wants to warm his hands quickly by rubb...,Mercury_SC_415702,George wants to warm his hands quickly by rubb...
3,A,D,palms covered with lotion,Glabrous skin covers the surface of the palms ...,George wants to warm his hands quickly by rubb...,Mercury_SC_415702,George wants to warm his hands quickly by rubb...
4,B,A,The refrigerator door is smooth.,An everyday example is a refrigerator magnet u...,The refrigerator door is smooth statements bes...,MCAS_2009_5_6516,Which of the following statements best explain...


In [35]:
df.shape

(4460, 7)

In [36]:
df.shape[0]/4

1115.0

In [29]:
list(df.columns)

['answerKey',
 'choice_label',
 'choice_text',
 'concat_supports',
 'hypothesis',
 'id',
 'question_stem']

In [37]:
columns = ['answerKey',
 'option_A',
 'option_B',
 'option_C',
 'option_D',
 'concat_supports_A',
 'concat_supports_B',
 'concat_supports_C',
 'concat_supports_D',
 'hypothesis_A',
 'hypothesis_B',
 'hypothesis_C',
 'hypothesis_D',
 'id',
 'question_stem']

In [38]:
df = pd.DataFrame(columns=columns)
df.to_csv("processed/Train_Final.csv", encoding='utf-8', index=False)
df.to_csv("processed/Test_Final.csv", encoding='utf-8', index=False)
df.to_csv("processed/Dev_Final.csv", encoding='utf-8', index=False)

In [42]:
df_dev.shape

(0, 15)

In [43]:
df_dev = pd.read_csv("processed/Dev_Final.csv")
df_dev.head()

,answerKey,option_A,option_B,option_C,option_D,concat_supports_A,concat_supports_B,concat_supports_C,concat_supports_D,hypothesis_A,hypothesis_B,hypothesis_C,hypothesis_D,id,question_stem


In [58]:
import csv

def processed_file(in_path, out_path):
    with open(in_path, mode = 'r') as csv_file_r:
        with open(out_path, mode = 'w') as csv_file_w:
            csv_reader = csv.reader(csv_file_r, delimiter=',')
            csv_writer = csv.writer(csv_file_w, delimiter=',')
            csv_writer.writerow(columns)
            temp_lst = []
            for i, row in enumerate(csv_reader):
                    if i == 0:
                        continue
                    if i%4 == 1:
                        # print(row[1])
                        assert row[1] == 'A','something wrong'   
                        temp_lst = ['']*15   
                        temp_lst[0] = row[0]
                        temp_lst[-2] = row[-2]
                        temp_lst[-1] = row[-1]
                        temp_lst[1] = row[2]
                        temp_lst[5] = row[3]
                        temp_lst[9] = row[4]
                    elif i%4 == 2:
                        assert row[1] == 'B','something wrong' 
                        assert row[-2] == temp_lst[-2],'everthing wrong'
                        temp_lst[2] = row[2]
                        temp_lst[6] = row[3]
                        temp_lst[10] = row[4]
                    elif i%4 ==3:
                        assert row[1] == 'C','something wrong' 
                        assert row[-2] == temp_lst[-2],'everthing wrong'
                        temp_lst[3] = row[2]
                        temp_lst[7] = row[3]
                        temp_lst[11] = row[4]
                    elif i%4 == 0:
                        assert row[1] == 'D','something wrong' 
                        assert row[-2] == temp_lst[-2],'everthing wrong'
                        temp_lst[4] = row[2]
                        temp_lst[8] = row[3]
                        temp_lst[12] = row[4]
                        csv_writer.writerow(temp_lst)
                
        

In [59]:
processed_file("processed/Dev.csv", "processed/Dev_Final.csv")


In [60]:
df_dev = pd.read_csv("processed/Dev_Final.csv")
df_dev.head()

,answerKey,option_A,option_B,option_C,option_D,concat_supports_A,concat_supports_B,concat_supports_C,concat_supports_D,hypothesis_A,hypothesis_B,hypothesis_C,hypothesis_D,id,question_stem
0,D,Put the objects in groups.,Change the height of the ramp.,Choose different objects to roll.,Record the details of the investigation.,Students explain why the objects roll down the...,Students explain why the objects roll down the...,Students explain why the objects roll down the...,Students investigate moving objects up a ramp....,Juan and LaKeisha roll a few objects down a ra...,Juan and LaKeisha roll a few objects down a ra...,Juan and LaKeisha roll a few objects down a ra...,Juan and LaKeisha roll a few objects down a ra...,Mercury_SC_407695,Juan and LaKeisha roll a few objects down a ra...
1,C,fog,rain,drought,tornado,The colder temperatures cause water vapor to c...,"High pressure causes air to sink, preventing c...",Definitions of meteorological drought must be ...,If a low pressure system remains for days afte...,High-pressure systems stop air from rising int...,High-pressure systems stop air from rising int...,High-pressure systems stop air from rising int...,High-pressure systems stop air from rising int...,Mercury_7103565,High-pressure systems stop air from rising int...
2,D,The sun revolves around Earth.,Earth rotates around the sun.,The sun revolves on its axis.,Earth rotates on its axis.,"This rotation causes the sun, moon, and stars ...","This rotation causes the sun, moon, and stars ...","This rotation causes the sun, moon, and stars ...","This rotation causes the sun, moon, and stars ...",Students visited the Morris W. Offit telescope...,Students visited the Morris W. Offit telescope...,Students visited the Morris W. Offit telescope...,Students visited the Morris W. Offit telescope...,MDSA_2009_5_16,Students visited the Morris W. Offit telescope...
3,A,converting sunlight into electricity,looking for new coal reserves,finding reservoirs that contain oil,converting forests into farmland,Converting Sunlight into Electricity. Amorphou...,He would have to find a new area of research. ...,Find an example of an environmental problem in...,The converting of forests to farmland would ha...,Converting sunlight into electricity topic are...,Looking for new coal reserves topic area would...,Finding reservoirs that contain oil topic area...,Converting forests into farmland topic area wo...,Mercury_7027230,Which topic area would be the best to research...
4,B,Shady areas increased.,Food sources increased.,Oxygen levels increased.,Available water increased.,When more acorns are produced by oak trees the...,When more acorns are produced by oak trees the...,When more acorns are produced by oak trees the...,When more acorns are produced by oak trees the...,"One year, the oak trees in a park began produc...","One year, the oak trees in a park began produc...","One year, the oak trees in a park began produc...","One year, the oak trees in a park began produc...",Mercury_SC_405487,"One year, the oak trees in a park began produc..."


In [61]:
df_dev.shape

(294, 15)

In [75]:
print(df_dev.loc[1]['option_A'])
print(df_dev.loc[1]['option_B'])
print(df_dev.loc[1]['option_C'])
print(df_dev.loc[1]['option_D'])

fog
rain
drought
tornado


In [76]:
print(df_dev.loc[1]['concat_supports_A'])
print(df_dev.loc[1]['concat_supports_B'])
print(df_dev.loc[1]['concat_supports_C'])
print(df_dev.loc[1]['concat_supports_D'])

The colder temperatures cause water vapor to condense into water droplets and can result in fog. Fog is formed when the air cools enough that the vapor pressure encourages condensation over evaporation . The usual form which the atmospheric moisture takes is that of a bank of fog, and as the condensation of vapour takes place more rapidlv in high latitudes, Anthelia are most likely to be observed in mountainous regions. Fog forms when the air has a high relative humidity, the water vapour condenses to form very small droplets, which remain suspended in the thermal currents of the air. Fog can also form overnight within a region of higher pressure. High blood pressure, candida, sleep disorder, bladder control and mental fog. The water vapor in the air contacting this cold water condenses into fog. Like clouds, fog is made up of condensed water droplets which are the result of the air being cooled to the point (actually, the dewpoint ) where it can no longer hold all of the water vapor i

In [78]:
print(df_dev.loc[1]['question_stem'])

High-pressure systems stop air from rising into the colder regions of the atmosphere where water can condense. What will most likely result if a high-pressure system remains in an area for a long period of time?


In [79]:
print(df_dev.loc[1]['hypothesis_A'])
print(df_dev.loc[1]['hypothesis_B'])
print(df_dev.loc[1]['hypothesis_C'])
print(df_dev.loc[1]['hypothesis_D'])

High-pressure systems stop air from rising into the colder regions of the atmosphere where water can condense. Fog will most likely result if a high-pressure system remains in an area for a long period of time.
High-pressure systems stop air from rising into the colder regions of the atmosphere where water can condense. Rain will most likely result if a high-pressure system remains in an area for a long period of time.
High-pressure systems stop air from rising into the colder regions of the atmosphere where water can condense. Drought will most likely result if a high-pressure system remains in an area for a long period of time.
High-pressure systems stop air from rising into the colder regions of the atmosphere where water can condense. Tornado will most likely result if a high-pressure system remains in an area for a long period of time.


In [62]:
processed_file("processed/Train.csv", "processed/Train_Final.csv")

In [63]:
df_train = pd.read_csv("processed/Train_Final.csv")
df_train.head()

,answerKey,option_A,option_B,option_C,option_D,concat_supports_A,concat_supports_B,concat_supports_C,concat_supports_D,hypothesis_A,hypothesis_B,hypothesis_C,hypothesis_D,id,question_stem
0,A,dry palms,wet palms,palms covered with oil,palms covered with lotion,Motion produces heat when we rub the palms of ...,Motion produces heat when we rub the palms of ...,Heat up the oil a bit in the microwave and rub...,Glabrous skin covers the surface of the palms ...,George wants to warm his hands quickly by rubb...,George wants to warm his hands quickly by rubb...,George wants to warm his hands quickly by rubb...,George wants to warm his hands quickly by rubb...,Mercury_SC_415702,George wants to warm his hands quickly by rubb...
1,B,The refrigerator door is smooth.,The refrigerator door contains iron.,The refrigerator door is a good conductor.,The refrigerator door has electric wires in it.,An everyday example is a refrigerator magnet u...,An everyday example is a refrigerator magnet u...,An everyday example is a refrigerator magnet u...,An everyday example is a refrigerator magnet u...,The refrigerator door is smooth statements bes...,The refrigerator door contains iron statements...,The refrigerator door is a good conductor stat...,The refrigerator door has electric wires in it...,MCAS_2009_5_6516,Which of the following statements best explain...
2,B,cooling of flowing magma.,converging of crustal plates.,deposition of river sediments.,solution of carbonate minerals.,Magma that intrudes between sedimentary rock l...,"along convergent plate boundaries, flat-lying ...",Sedimentary rocks are deposited into layers. E...,Sedimentary rock that forms when minerals crys...,A fold observed in layers of sedimentary rock ...,A fold observed in layers of sedimentary rock ...,A fold observed in layers of sedimentary rock ...,A fold observed in layers of sedimentary rock ...,Mercury_7233695,A fold observed in layers of sedimentary rock ...
3,D,worldwide disease,global mountain building,rise of mammals that preyed upon plants and an...,impact of an asteroid created dust that blocke...,The most recent group of Mesozoic sea reptiles...,As mountain building increased toward the end ...,The Paleozoic Era saw the development of many ...,Most researchers agree that the Mesozoic Era e...,Worldwide disease do scientists offer as the m...,Global mountain building do scientists offer a...,Rise of mammals that preyed upon plants and an...,Impact of an asteroid created dust that blocke...,Mercury_7041615,Which of these do scientists offer as the most...
4,B,west,east,north,south,A 10 mile race is held: the boats sail downstr...,A 10 mile race is held: the boats sail downstr...,Trade winds north of the equator blow from the...,The boat was sailing into the wind and veering...,A boat is acted on by a river current flowing ...,A boat is acted on by a river current flowing ...,A boat is acted on by a river current flowing ...,A boat is acted on by a river current flowing ...,Mercury_7041860,A boat is acted on by a river current flowing ...


In [64]:
df_train.shape

(1115, 15)

In [80]:
print(df_train.loc[11]['option_A'])
print(df_train.loc[11]['option_B'])
print(df_train.loc[11]['option_C'])
print(df_train.loc[11]['option_D'])

lighting a match
breaking a glass
burning of gasoline
rusting of iron


In [81]:
print(df_test.loc[11]['concat_supports_A'])
print(df_test.loc[11]['concat_supports_B'])
print(df_test.loc[11]['concat_supports_C'])
print(df_test.loc[11]['concat_supports_D'])

The author first describes the effect of the sun on the atmosphere and the creation of wind and waves, including the effects of global warming. The ocean harbors abundant energy in the form of wind, waves and sun. Sun Light: Grow best in full sun. The sun waves radiate the short-wave radiation . In astrol., horn under the predominant in- fluence of the sun ; influenced by the sun. motion of the Sun and Moon influences. Make a bold statement in the sun or shade. This is because it is based not on the sun or the wind or the waves, but on the temperature difference between warm water at the sun-heated surface and cold water in the deep, dark ocean. Key words: magnetohydrodynamics (MHD) -- waves -- Sun: oscillations -- Sun: magnetic fields. Tides are the regular rise and fall in water level experienced by seas and oceans in response to the gravitational influences of the Moon and the Sun, and the effects of the Earth's rotation. Allow for a minimum of six hours of sun a day for best foliag

In [82]:
print(df_train.loc[11]['question_stem'])

Which of the following is an example of a physical change?


In [83]:
print(df_train.loc[11]['hypothesis_A'])
print(df_train.loc[11]['hypothesis_B'])
print(df_train.loc[11]['hypothesis_C'])
print(df_train.loc[11]['hypothesis_D'])

Lighting a match is an example of a physical change.
Breaking a glass is an example of a physical change.
Burning of gasoline is an example of a physical change.
Rusting of iron is an example of a physical change.


In [65]:
processed_file("processed/Test.csv", "processed/Test_Final.csv")

In [66]:
df_test = pd.read_csv("processed/Test_Final.csv")
df_test.head()

,answerKey,option_A,option_B,option_C,option_D,concat_supports_A,concat_supports_B,concat_supports_C,concat_supports_D,hypothesis_A,hypothesis_B,hypothesis_C,hypothesis_D,id,question_stem
0,C,Planetary density will decrease.,Planetary years will become longer.,Planetary days will become shorter.,Planetary gravity will become stronger.,As latitude increases and the speed of the ear...,Four-year rotation or longer. Two-year rotatio...,Planetary rotation and orbit. Identify the pla...,Planetary rotation and orbit. If the body does...,An astronomer observes that a planet rotates f...,An astronomer observes that a planet rotates f...,An astronomer observes that a planet rotates f...,An astronomer observes that a planet rotates f...,Mercury_7175875,An astronomer observes that a planet rotates f...
1,B,buildings will be built faster,buildings will be made safer,building designs will look nicer,building materials will be cheaper,Newer buildings are designed to withstand even...,Newer buildings are designed to withstand even...,Newer buildings are designed to withstand even...,Newer buildings are designed to withstand even...,A group of engineers wanted to know how differ...,A group of engineers wanted to know how differ...,A group of engineers wanted to know how differ...,A group of engineers wanted to know how differ...,Mercury_SC_409171,A group of engineers wanted to know how differ...
2,C,Chemical energy is absorbed through the roots.,Light energy is converted to chemical energy.,Chlorophyll in the leaf captures light energy.,Sunlight is converted into chlorophyll.,The process of photosynthesis converts light i...,The process of photosynthesis converts light i...,Photosynthesis requires chlorophyll to capture...,Plants release oxygen as a by-product of photo...,The end result in the process of photosynthesi...,The end result in the process of photosynthesi...,The end result in the process of photosynthesi...,The end result in the process of photosynthesi...,Mercury_SC_408547,The end result in the process of photosynthesi...
3,D,the control,the hypothesis statement,the dependent (responding) variable,the independent (manipulated) variable,Plus the three speed electronic speed controll...,Astronomer and physicist Tim Thompson has crit...,A speed is defined as how a variable changes p...,Independent speed calculations suggest that a ...,A physicist wants to determine the speed a car...,A physicist wants to determine the speed a car...,A physicist wants to determine the speed a car...,A physicist wants to determine the speed a car...,Mercury_407327,A physicist wants to determine the speed a car...
4,D,They have each lost kinetic energy.,They have each gained the same amount of poten...,They have each lost the same amount of potenti...,They have each gained one-half of their maximu...,Each object has a mass of 1.0 kg . The mass of...,Each object has a mass of 1.0 kg . The mass of...,Each object has a mass of 1.0 kg . The mass of...,Each object has a mass of 1.0 kg . The mass of...,An astronaut drops a 1.0 kg object and a 5.0 k...,An astronaut drops a 1.0 kg object and a 5.0 k...,An astronaut drops a 1.0 kg object and a 5.0 k...,An astronaut drops a 1.0 kg object and a 5.0 k...,MCAS_2006_9_44,An astronaut drops a 1.0 kg object and a 5.0 k...


In [67]:
df_test.shape

(1161, 15)

In [84]:
print(df_test.loc[10]['option_A'])
print(df_test.loc[10]['option_B'])
print(df_test.loc[10]['option_C'])
print(df_test.loc[10]['option_D'])

life processes
size differences
plasma membranes
energy molecules


In [85]:
print(df_test.loc[10]['concat_supports_A'])
print(df_test.loc[10]['concat_supports_B'])
print(df_test.loc[10]['concat_supports_C'])
print(df_test.loc[10]['concat_supports_D'])

Prepared from virus, prokaryotic cell, or eukaryotic cell culturing process:. Prokaryotic and Eukaryotic Cells Cells are the basic building block of life. However, the process of cell division in prokaryotes and in single-celled eukaryotes differs. These simple prokaryotic cells actually merged to form the first eukaryotic cells by a process. A process by which the eukaryotic cell's endomembrane system evolved from inward folds of the plasma membrane of a prokaryotic cell. process which eukaryotic cell engulfs another living prokaryote. Despite differences between prokaryotes and eukaryotes, there are several common features in their cell division processes. Although both prokaryotic and eukaryotic cells could be used as input into the process, only eukaryotic cells would emerge as output of the process, because of the ability of their cell membranes to self assemble. unit of life TWO BASIC CELL TYPES EUKARYOTIC CELL PROKARYOTIC CELL EUKARYOTIC CELL Cell which has a true nucleus and su

In [86]:
print(df_test.loc[10]['question_stem'])

According to cell classification, prokaryotic cells are separated from eukaryotic cells. Which feature is often used to distinguish prokaryotic cells from eukaryotic cells?


In [87]:
print(df_test.loc[10]['hypothesis_A'])
print(df_test.loc[10]['hypothesis_B'])
print(df_test.loc[10]['hypothesis_C'])
print(df_test.loc[10]['hypothesis_D'])

According to cell classification, prokaryotic cells are separated from eukaryotic cells. Life processes feature is often used to distinguish prokaryotic cells from eukaryotic cells.
According to cell classification, prokaryotic cells are separated from eukaryotic cells. Size differences feature is often used to distinguish prokaryotic cells from eukaryotic cells.
According to cell classification, prokaryotic cells are separated from eukaryotic cells. Plasma membranes feature is often used to distinguish prokaryotic cells from eukaryotic cells.
According to cell classification, prokaryotic cells are separated from eukaryotic cells. Energy molecules feature is often used to distinguish prokaryotic cells from eukaryotic cells.


In [32]:
processed_file("processed/Train.csv", "processed/Dev_Final.csv")

0 id
1 Mercury_SC_415702
2 Mercury_SC_415702
3 Mercury_SC_415702
4 Mercury_SC_415702
5 MCAS_2009_5_6516
6 MCAS_2009_5_6516
7 MCAS_2009_5_6516
8 MCAS_2009_5_6516
9 Mercury_7233695
10 Mercury_7233695
11 Mercury_7233695
12 Mercury_7233695
13 Mercury_7041615
14 Mercury_7041615
15 Mercury_7041615
16 Mercury_7041615
17 Mercury_7041860
18 Mercury_7041860
19 Mercury_7041860
20 Mercury_7041860
21 Mercury_SC_401653
22 Mercury_SC_401653
23 Mercury_SC_401653
24 Mercury_SC_401653
25 MEA_2016_8_14
26 MEA_2016_8_14
27 MEA_2016_8_14
28 MEA_2016_8_14
29 ACTAAP_2013_5_11
30 ACTAAP_2013_5_11
31 ACTAAP_2013_5_11
32 ACTAAP_2013_5_11
33 MCAS_1998_4_3
34 MCAS_1998_4_3
35 MCAS_1998_4_3
36 MCAS_1998_4_3
37 Mercury_7106908
38 Mercury_7106908
39 Mercury_7106908
40 Mercury_7106908
41 Mercury_401402
42 Mercury_401402
43 Mercury_401402
44 Mercury_401402
45 MCAS_2005_8_4
46 MCAS_2005_8_4
47 MCAS_2005_8_4
48 MCAS_2005_8_4
49 MCAS_2007_8_5180
50 MCAS_2007_8_5180
51 MCAS_2007_8_5180
52 MCAS_2007_8_5180
53 TIMSS_1995_8_

539 MCAS_2016_8_10
540 MCAS_2016_8_10
541 Mercury_412782
542 Mercury_412782
543 Mercury_412782
544 Mercury_412782
545 TIMSS_2003_8_pg52
546 TIMSS_2003_8_pg52
547 TIMSS_2003_8_pg52
548 TIMSS_2003_8_pg52
549 Mercury_SC_405796
550 Mercury_SC_405796
551 Mercury_SC_405796
552 Mercury_SC_405796
553 ACTAAP_2008_5_6
554 ACTAAP_2008_5_6
555 ACTAAP_2008_5_6
556 ACTAAP_2008_5_6
557 Mercury_7239418
558 Mercury_7239418
559 Mercury_7239418
560 Mercury_7239418
561 Mercury_7222355
562 Mercury_7222355
563 Mercury_7222355
564 Mercury_7222355
565 Mercury_7215198
566 Mercury_7215198
567 Mercury_7215198
568 Mercury_7215198
569 Mercury_7179778
570 Mercury_7179778
571 Mercury_7179778
572 Mercury_7179778
573 Mercury_7093310
574 Mercury_7093310
575 Mercury_7093310
576 Mercury_7093310
577 Mercury_7099365
578 Mercury_7099365
579 Mercury_7099365
580 Mercury_7099365
581 VASoL_2007_3_35
582 VASoL_2007_3_35
583 VASoL_2007_3_35
584 VASoL_2007_3_35
585 Mercury_7016818
586 Mercury_7016818
587 Mercury_7016818
588 Mercur

1346 MCAS_2014_5_4
1347 MCAS_2014_5_4
1348 MCAS_2014_5_4
1349 Mercury_7097388
1350 Mercury_7097388
1351 Mercury_7097388
1352 Mercury_7097388
1353 Mercury_7128380
1354 Mercury_7128380
1355 Mercury_7128380
1356 Mercury_7128380
1357 MDSA_2011_8_25
1358 MDSA_2011_8_25
1359 MDSA_2011_8_25
1360 MDSA_2011_8_25
1361 Mercury_416466
1362 Mercury_416466
1363 Mercury_416466
1364 Mercury_416466
1365 Mercury_7218155
1366 Mercury_7218155
1367 Mercury_7218155
1368 Mercury_7218155
1369 MSA_2015_8_31
1370 MSA_2015_8_31
1371 MSA_2015_8_31
1372 MSA_2015_8_31
1373 ACTAAP_2009_7_14
1374 ACTAAP_2009_7_14
1375 ACTAAP_2009_7_14
1376 ACTAAP_2009_7_14
1377 Mercury_7114818
1378 Mercury_7114818
1379 Mercury_7114818
1380 Mercury_7114818
1381 MCAS_2003_5_6
1382 MCAS_2003_5_6
1383 MCAS_2003_5_6
1384 MCAS_2003_5_6
1385 Mercury_SC_400607
1386 Mercury_SC_400607
1387 Mercury_SC_400607
1388 Mercury_SC_400607
1389 Mercury_7210263
1390 Mercury_7210263
1391 Mercury_7210263
1392 Mercury_7210263
1393 Mercury_SC_416424
1394 Mer

2539 Mercury_7038868
2540 Mercury_7038868
2541 MDSA_2008_5_16
2542 MDSA_2008_5_16
2543 MDSA_2008_5_16
2544 MDSA_2008_5_16
2545 ACTAAP_2010_5_14
2546 ACTAAP_2010_5_14
2547 ACTAAP_2010_5_14
2548 ACTAAP_2010_5_14
2549 Mercury_182018
2550 Mercury_182018
2551 Mercury_182018
2552 Mercury_182018
2553 Mercury_7122570
2554 Mercury_7122570
2555 Mercury_7122570
2556 Mercury_7122570
2557 Mercury_7188090
2558 Mercury_7188090
2559 Mercury_7188090
2560 Mercury_7188090
2561 NCEOGA_2013_5_42
2562 NCEOGA_2013_5_42
2563 NCEOGA_2013_5_42
2564 NCEOGA_2013_5_42
2565 Mercury_7016730
2566 Mercury_7016730
2567 Mercury_7016730
2568 Mercury_7016730
2569 MDSA_2008_5_18
2570 MDSA_2008_5_18
2571 MDSA_2008_5_18
2572 MDSA_2008_5_18
2573 NYSEDREGENTS_2016_4_20
2574 NYSEDREGENTS_2016_4_20
2575 NYSEDREGENTS_2016_4_20
2576 NYSEDREGENTS_2016_4_20
2577 Mercury_409819
2578 Mercury_409819
2579 Mercury_409819
2580 Mercury_409819
2581 MCAS_2006_9_21
2582 MCAS_2006_9_21
2583 MCAS_2006_9_21
2584 MCAS_2006_9_21
2585 Mercury_72457

3906 Mercury_7230545
3907 Mercury_7230545
3908 Mercury_7230545
3909 Mercury_7108833
3910 Mercury_7108833
3911 Mercury_7108833
3912 Mercury_7108833
3913 MEA_2012_5_8
3914 MEA_2012_5_8
3915 MEA_2012_5_8
3916 MEA_2012_5_8
3917 MDSA_2008_8_26
3918 MDSA_2008_8_26
3919 MDSA_2008_8_26
3920 MDSA_2008_8_26
3921 Mercury_7142975
3922 Mercury_7142975
3923 Mercury_7142975
3924 Mercury_7142975
3925 MEAP_2005_5_38
3926 MEAP_2005_5_38
3927 MEAP_2005_5_38
3928 MEAP_2005_5_38
3929 Mercury_SC_415468
3930 Mercury_SC_415468
3931 Mercury_SC_415468
3932 Mercury_SC_415468
3933 Mercury_7102340
3934 Mercury_7102340
3935 Mercury_7102340
3936 Mercury_7102340
3937 Mercury_7033583
3938 Mercury_7033583
3939 Mercury_7033583
3940 Mercury_7033583
3941 Mercury_400350
3942 Mercury_400350
3943 Mercury_400350
3944 Mercury_400350
3945 Mercury_7145495
3946 Mercury_7145495
3947 Mercury_7145495
3948 Mercury_7145495
3949 NCEOGA_2013_5_60
3950 NCEOGA_2013_5_60
3951 NCEOGA_2013_5_60
3952 NCEOGA_2013_5_60
3953 Mercury_7038150
3954

In [ ]:
def set_annotation_labels(obj):
    if obj["answerKey"] == obj["question"]["choice"]["label"]:
        return ["entailment"]
    else:
        return ["contradiction"]
    

In [ ]:
def transform_to_snli(in_file_name, out_file_name):
    dict_list = []
    with jsonlines.open(in_file_name) as reader:
        with jsonlines.open(out_file_name, mode='w') as writer:
            for obj in reader:
                temp_dict = {}
                temp_dict["annotator_labels"] = set_annotation_labels(obj)
                temp_dict["captionID"] = obj["id"]
                temp_dict["gold_label"] = set_annotation_labels(obj)[0]
                temp_dict["pairID"] = obj["id"]
                temp_dict["sentence1"] = get_concat_supports(obj['supports'])
                temp_dict["sentence1_binary_parse"] = binarize(parse_sentence(temp_dict["sentence1"]))
                temp_dict["sentence1_parse"] = ""
                temp_dict["sentence2"] = obj["hypothesis"]
                temp_dict["sentence2_binary_parse"] = binarize(parse_sentence(obj["hypothesis"]))
                temp_dict["sentence2_parse"] = ""
                writer.write(temp_dict)

In [ ]:
transform_to_snli("3_ARC-Challenge_Dev_openie.jsonl", "ARC_Dev_openie_snli_format.jsonl")

In [ ]:
with jsonlines.open("3_ARC-Challenge_Dev_openie.jsonl", 'r') as reader:
    for obj in reader:
        print(type(obj['supports']))
        print(obj['supports'])
        print(get_concat_supports(obj['supports']))
        break

In [ ]:
COLUMN_NAMES = ["id", "question_stem", "choice_text", "choice_label", "answerKey", "hypothesis", "concat_supports"]

In [ ]:
create_csv("3_ARC-Challenge_Dev_openie.jsonl", "3_ARC-Challenge_Dev_openie.csv")

In [ ]:
create_csv("3_ARC-Challenge_Test_openie.jsonl", "3_ARC-Challenge_Test_openie.csv")

In [ ]:
create_csv("3_ARC-Challenge_Train_openie.jsonl", "3_ARC-Challenge_Train_openie.csv")

In [ ]:
import re
import string
from stanfordcorenlp import StanfordCoreNLP
from nltk import Tree
from functools import reduce
regex = re.compile('[%s]' % re.escape(string.punctuation))

def parse_sentence(sentence):
    nlp = StanfordCoreNLP('http://localhost', port=9000, timeout=30000)
    sentence = regex.sub('', sentence)

    result = nlp.parse(sentence)
    result = result.replace('\n', '')
    result = re.sub(' +',' ', result)

    nlp.close() # Do not forget to close! The backend server will consume a lot memery.
    return result # result.encode("utf-8")

def binarize(parsed_sentence):
    # print(type(parsed_sentence))
    sentence = parsed_sentence.replace("\n", "")

    for pattern in ["ROOT", "SINV", "NP", "S", "PP", "ADJP", "SBAR", 
                    "DT", "JJ", "NNS", "VP", "VBP", "RB"]:
        sentence = sentence.replace("({}".format(pattern), "(")

    sentence = re.sub(' +',' ', sentence)
    # print(type(sentence))
    return sentence

In [ ]:
print(binarize(parse_sentence("The new rights are nice enough.")))